# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ola,59.5833,151.2833,8.68,87,100,0.61,RU,1718672065
1,1,adamstown,-25.0660,-130.1015,22.62,83,100,6.40,PN,1718672066
2,2,lihue,21.9789,-159.3672,30.58,69,75,6.17,US,1718672066
3,3,margaret river,-33.9500,115.0667,15.20,73,98,7.76,AU,1718672066
4,4,colonia,50.9333,6.9500,16.09,90,0,1.03,DE,1718672066


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,howard springs,-12.4970,131.0470,26.60,64,0,4.36,AU,1718672072
99,99,doda,33.1333,75.5667,23.54,52,0,2.32,IN,1718672081
103,103,bom jesus,-9.0744,-44.3586,21.08,52,0,0.53,BR,1718672051
114,114,judeida makr,32.9256,35.1550,22.97,85,0,0.76,IL,1718672083
138,138,zharkent,44.1628,80.0000,22.47,40,0,2.35,KZ,1718672086
141,141,alvorada,-12.4800,-49.1247,21.99,43,0,2.00,BR,1718672087
247,247,omerli,37.4022,40.9541,22.60,15,0,1.60,TR,1718672104
275,275,dryden,49.7833,-92.7503,22.16,33,0,3.09,CA,1718672109
313,313,darwin,-12.4611,130.8418,26.64,49,0,4.12,AU,1718671946
359,359,tamanrasset,22.7850,5.5228,26.95,10,0,3.09,DZ,1718672122


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
}
hotel_df=pd.DataFrame(hotel_dict)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
46,howard springs,AU,-12.4970,131.0470,
99,doda,IN,33.1333,75.5667,
103,bom jesus,BR,-9.0744,-44.3586,
114,judeida makr,IL,32.9256,35.1550,
138,zharkent,KZ,44.1628,80.0000,
141,alvorada,BR,-12.4800,-49.1247,
247,omerli,TR,37.4022,40.9541,
275,dryden,CA,49.7833,-92.7503,
313,darwin,AU,-12.4611,130.8418,
359,tamanrasset,DZ,22.7850,5.5228,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
howard springs - nearest hotel: Rydges Palmerston
doda - nearest hotel: No hotel found
bom jesus - nearest hotel: Hotel Familiar
judeida makr - nearest hotel: Akko Beach Hotel
zharkent - nearest hotel: Hotel (good)
alvorada - nearest hotel: No hotel found
omerli - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
darwin - nearest hotel: Mantra Pantanas Darwin
tamanrasset - nearest hotel: فندق الأمان
mashhad - nearest hotel: هتل تعاون
americus - nearest hotel: 8 Inn
mesolongi - nearest hotel: AVRA
al bawiti - nearest hotel: Old Oasis Hotel
taro - nearest hotel: Padma Resort Ubud


,City,Country,Lat,Lng,Hotel Name
46,howard springs,AU,-12.4970,131.0470,Rydges Palmerston
99,doda,IN,33.1333,75.5667,No hotel found
103,bom jesus,BR,-9.0744,-44.3586,Hotel Familiar
114,judeida makr,IL,32.9256,35.1550,Akko Beach Hotel
138,zharkent,KZ,44.1628,80.0000,Hotel (good)
141,alvorada,BR,-12.4800,-49.1247,No hotel found
247,omerli,TR,37.4022,40.9541,No hotel found
275,dryden,CA,49.7833,-92.7503,Holiday Inn Express
313,darwin,AU,-12.4611,130.8418,Mantra Pantanas Darwin
359,tamanrasset,DZ,22.7850,5.5228,فندق الأمان


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)